In [117]:
import tensorflow as tf # tensorflow 2.0
import numpy as np
from os import listdir
from os.path import isfile, join
from sklearn.model_selection import train_test_split
import imageio
import pandas as pd
import keras
from sklearn.metrics import classification_report, confusion_matrix
from matplotlib import pyplot as plt
from PIL import Image
import glob
import numpy as np
from sklearn.model_selection import train_test_split
import pandas as pd

In [118]:
labels = pd.read_csv("../../Xray_TeethLabels_Simple.csv",index_col=0)
Ylabels = labels[5:]
metaY = labels[:3]

In [119]:
X = []
Y = []
Z = []
for picFilename in glob.glob("../../processed2/processed/*"):
    #print(picFilename)
    patNumber = picFilename.split('/')[4].split('_')[0]
    toothNumber = picFilename.split('/')[4].split('_')[1].split('.')[0]
    #print(Ylabels.loc[toothNumber,patNumber])
    im = Image.open(picFilename)
    X.append(np.array(im))
    if (Ylabels.loc[toothNumber,patNumber] == "Yes"):
        Y.append(1)
    else:
        Y.append(0)
    Z.append(picFilename)
X = np.array(X)

X_train, X_test, Y_train_original, Y_test_original,Z_train,Z_test = train_test_split(X, np.array(Y),Z, random_state = 100, test_size=0.20)

Y_train = tf.keras.utils.to_categorical(Y_train_original, 2)
Y_test = tf.keras.utils.to_categorical(Y_test_original, 2)

In [120]:
print(len(X))
print(len(np.column_stack((Z_train,Y_train))))

224
179


In [121]:
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten
from keras.layers import MaxPooling2D, Dropout

model = Sequential()
model.add(Conv2D(256, 5, activation='relu', input_shape=(500, 500, 3)))
#model.add(Dropout(0.05))
model.add(MaxPooling2D())

model.add(Conv2D(128, 5, activation='relu'))

model.add(MaxPooling2D())
model.add(Dropout(0.5))

model.add(Flatten())
model.add(Dense(1024, activation='relu'))
#model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [122]:
max_epochs = 3
model.fit(X_train, Y_train, validation_data=(X_test, Y_test), epochs=max_epochs)
info = model.evaluate(X_test, Y_test)
print('Accuracy score for epoches ' + str(max_epochs) + ':', info[1])

predictions = (model.predict(X_test) > 0.5).astype(int)
print(predictions)
pred = []

for prediction in predictions:
    i, = np.where(prediction == 1)
    pred.append(i[0])

print(pred)
print(Y_test_original)
print(classification_report(Y_test_original, pred))
tn, fp, fn, tp = confusion_matrix(Y_test_original, pred).ravel()
print(tn, fp, fn, tp)


Epoch 1/3
6/6 [==============================] - 363s 56s/step - loss: 24320.1895 - accuracy: 0.5642 - val_loss: 59.7711 - val_accuracy: 0.5111
Epoch 2/3
6/6 [==============================] - 286s 47s/step - loss: 8.6238 - accuracy: 0.5866 - val_loss: 0.6632 - val_accuracy: 0.6000
Epoch 3/3
2/2 [==============================] - 15s 4s/step - loss: 0.9745 - accuracy: 0.6444
Accuracy score for epoches 3: 0.644444465637207
[[1 0]
 [1 0]
 [0 1]
 [0 1]
 [1 0]
 [0 1]
 [1 0]
 [0 1]
 [1 0]
 [1 0]
 [1 0]
 [0 1]
 [1 0]
 [0 1]
 [1 0]
 [1 0]
 [0 1]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [0 1]
 [1 0]
 [1 0]
 [0 1]
 [0 1]
 [1 0]
 [1 0]
 [0 1]
 [1 0]
 [0 1]
 [0 1]
 [1 0]
 [1 0]
 [1 0]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [1 0]
 [0 1]]
[0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1]
[1 0 0 0 1 1 0 0 0 0 0 1 1 1 0 0 0 0 1 0 1 0 0 0 1 1 1 0 1 1 1 0 0 1 1 1 0
 0 1 1 1 1 1 0 0]
              precision    re

In [123]:
image = X_test[4]
print(image.shape)
np.column_stack((Z_test,Y_test))

(500, 500, 3)


array([['../../processed2/processed/Patient 2_19.png', '0.0', '1.0'],
       ['../../processed2/processed/Patient 22_19.png', '1.0', '0.0'],
       ['../../processed2/processed/Patient 26_21.png', '1.0', '0.0'],
       ['../../processed2/processed/Patient 2_21.png', '1.0', '0.0'],
       ['../../processed2/processed/Patient 19_3.png', '0.0', '1.0'],
       ['../../processed2/processed/Patient 11_13.png', '0.0', '1.0'],
       ['../../processed2/processed/Patient 22_14.png', '1.0', '0.0'],
       ['../../processed2/processed/Patient 5_3.png', '1.0', '0.0'],
       ['../../processed2/processed/Patient 7_29.png', '1.0', '0.0'],
       ['../../processed2/processed/Patient 7_31.png', '1.0', '0.0'],
       ['../../processed2/processed/Patient 19_31.png', '1.0', '0.0'],
       ['../../processed2/processed/Patient 14_13.png', '0.0', '1.0'],
       ['../../processed2/processed/Patient 2_14.png', '0.0', '1.0'],
       ['../../processed2/processed/Patient 11_15.png', '0.0', '1.0'],
       ['../..

In [124]:
%load_ext autoreload
%autoreload 2
import os,sys
import lime
from lime import lime_image

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [125]:
explainer = lime_image.LimeImageExplainer()

In [126]:
explanation = explainer.explain_instance(image.astype('double'), model.predict, top_labels=2, hide_color=0, num_samples=100)

  0%|          | 0/1000 [00:00<?, ?it/s]

In [127]:
from skimage.segmentation import mark_boundaries

In [128]:
temp, mask = explanation.get_image_and_mask(explanation.top_labels[0], positive_only=False, num_features=10, hide_rest=True)
plt.imshow(mark_boundaries(temp, mask))

Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


In [129]:
plt.imshow(image)